In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, desc, count
spark = SparkSession.builder.appName('trial_mini_project').getOrCreate()



# importing data from datastet

df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)
#df_q2 = df.select('Dest', 'TailNum').where(df["Dest"] != '0')

df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- C

# 1) Find the most frequent tail number which is getting in destination 


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, desc,expr

#importing data from local data folder
df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)

#selecting the required columns , this makes processing faster
df_q2 = df.select('Dest','TailNum')

df_q2_no_null = df_q2.filter((df_q2["TailNum"] != "0") & (df_q2["TailNum"] != "000000")) #removing rows with TailNum = 0/000000

print(type(df_q2))
print(" +++++ "*10)
most_frequent = df_q2_no_null.groupBy("TailNum",'Dest').agg(count("TailNum").alias("Count_TailNum")).sort(col("Count_TailNum").desc()).show()

#most_frequent.distinct().show()

<class 'pyspark.sql.dataframe.DataFrame'>
 +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++ 
+-------+----+-------------+
|TailNum|Dest|Count_TailNum|
+-------+----+-------------+
| N655BR| HNL|         2241|
| N651BR| HNL|         2173|
| N654BR| HNL|         2138|
| N693BR| HNL|         2067|
| N479HA| HNL|         2038|
| N478HA| HNL|         2024|
| N485HA| HNL|         1984|
| N480HA| HNL|         1976|
| N484HA| HNL|         1944|
| N487HA| HNL|         1909|
| N481HA| HNL|         1868|
| N810AL| HNL|         1843|
| N477HA| HNL|         1837|
| N837AL| HNL|         1836|
| N475HA| HNL|         1816|
| N486HA| HNL|         1787|
| N646BR| HNL|         1768|
| N836AL| HNL|         1761|
| N824AL| HNL|         1754|
| N808AL| HNL|         1740|
+-------+----+-------------+
only showing top 20 rows



# 2) Find out the cancelled flight  details for the last quarter of the year 2007



In [3]:
df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)

df.na.drop()

df.filter((df.Month >= 8) & (df.Month <=12) & (df.Cancelled == 1)).show(5)


+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2007|    8|         2|        4|     NA|       830|     NA|       930|           WN|        7

# 3) Find out the average weather delays for a particular flight per month


In [4]:
from pyspark.sql.functions import avg, col, desc

df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)

df.groupBy("Month").avg('WeatherDelay').sort(col("Month")).show()

+-----+-------------------+
|Month|  avg(WeatherDelay)|
+-----+-------------------+
|    1| 0.8126742594025668|
|    2| 1.1426651862433788|
|    3| 0.6333765638468795|
|    4|   0.51643216930666|
|    5| 0.6052272846017077|
|    6| 1.2763936562420544|
|    7| 1.0766004687307265|
|    8| 0.8375915956275956|
|    9|0.41135346150449775|
|   10|0.45674389516057345|
|   11| 0.3357768086867862|
|   12| 1.1352771929481762|
+-----+-------------------+



# 4) Inspite of NASDelay, SecurityDelay, LateAircraftDelay,Weatherdealy which flight reached on time



In [5]:
df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)




df.filter(((df.NASDelay > 0) | (df.SecurityDelay > 0) | (df.LateAircraftDelay > 0) | (df.WeatherDelay > 0)) & (df.ArrDelay <= 0) ).count()

0

# 5) Month wise total distance travelled by each flight number in every month

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, desc


df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)
df_q2 = df.select('Month','FlightNum', 'Distance').filter((df["TailNum"] != "0") & (df["TailNum"] != "000000"))

print(type(df_q2))

total_distance = df_q2.groupBy("Month","FlightNum").sum("Distance").sort(col("Month")).show()

#total_count = df_q2.groupBy("TailNum").sum("Distance").sort(col("sum(Distance)").desc()).show()





<class 'pyspark.sql.dataframe.DataFrame'>
+-----+---------+-------------+
|Month|FlightNum|sum(Distance)|
+-----+---------+-------------+
|    1|     2383|        64128|
|    1|      285|       394434|
|    1|       95|       339349|
|    1|      690|       193603|
|    1|     1867|        74379|
|    1|     2645|        53660|
|    1|      534|       221538|
|    1|     2016|        81014|
|    1|     1528|       156561|
|    1|     2057|        19793|
|    1|      691|       209452|
|    1|     2513|        22538|
|    1|      670|       151788|
|    1|      463|        63309|
|    1|     1423|       164350|
|    1|     1871|        91504|
|    1|     1705|       124020|
|    1|     2017|        51207|
|    1|     2565|        11932|
|    1|     3253|         3355|
+-----+---------+-------------+
only showing top 20 rows



# 6) Month wise how many flights get diverted(origin to destination)



In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, desc


df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)
df_q2 = df.select('Origin','Month','Dest','TailNum','Diverted').filter((df["TailNum"] != "0") & (df["TailNum"] != "000000"))

print(" ++++ "*10)

# monthwise total diveted flight
total_sum_diverted = df_q2.groupBy("Month").sum("Diverted").sort(col("Month")).show()

print(" ++++ "*10)

# monthwise total diveted flight with respect to each origin and destination
total_distance = df_q2.groupBy("Month",'Origin','Dest').sum("Diverted").sort(col("Month")).show(12)

print(" ++++ "*10)

 ++++  ++++  ++++  ++++  ++++  ++++  ++++  ++++  ++++  ++++ 
+-----+-------------+
|Month|sum(Diverted)|
+-----+-------------+
|    1|         1200|
|    2|         1261|
|    3|         1275|
|    4|         1193|
|    5|         1442|
|    6|         2199|
|    7|         2150|
|    8|         2100|
|    9|          942|
|   10|          977|
|   11|          845|
|   12|         1488|
+-----+-------------+

 ++++  ++++  ++++  ++++  ++++  ++++  ++++  ++++  ++++  ++++ 
+-----+------+----+-------------+
|Month|Origin|Dest|sum(Diverted)|
+-----+------+----+-------------+
|    1|   ABQ| MCO|            0|
|    1|   ABQ| SEA|            0|
|    1|   BNA| DTW|            0|
|    1|   HOU| AUS|            2|
|    1|   SJC| BUR|            0|
|    1|   EWR| RDU|            0|
|    1|   SHV| IAH|            0|
|    1|   EWR| CLT|            1|
|    1|   MSY| EWR|            0|
|    1|   PHX| ASE|            3|
|    1|   CLE| CVG|            0|
|    1|   CAK| ATL|            0|
+-----+------+-

# 7) Week and month wise number of trips in all the flights



In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, desc


df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)
df_no_cancel = df.filter('Cancelled== 0') 
df_q2 = df_no_cancel.select('Month','DayOfWeek','TailNum').filter((df["TailNum"] != "0") & (df["TailNum"] != "000000") & (df["Cancelled"] == 0))



total_distance = df_q2.groupBy("Month",'DayOfWeek','TailNum').agg(count("TailNum").alias("total_trips")).sort(['Month','DayOfWeek']).show()

+-----+---------+-------+-----------+
|Month|DayOfWeek|TailNum|total_trips|
+-----+---------+-------+-----------+
|    1|        1| N511UA|         14|
|    1|        1| N434YV|         34|
|    1|        1| N16170|         21|
|    1|        1| N443CA|         21|
|    1|        1| N495CA|         29|
|    1|        1| N708CA|         22|
|    1|        1| N929LR|         30|
|    1|        1| N829UA|         19|
|    1|        1| N680AW|         23|
|    1|        1| N813AW|         24|
|    1|        1| N625AW|         22|
|    1|        1| N627AW|         13|
|    1|        1| N752UW|         26|
|    1|        1| N601DL|         15|
|    1|        1| N503AE|         28|
|    1|        1| N513AE|         25|
|    1|        1| N620NW|         17|
|    1|        1| N508US|         13|
|    1|        1| N583AA|         21|
|    1|        1| N958AS|         27|
+-----+---------+-------+-----------+
only showing top 20 rows



In [9]:
#total_distance.sort(['Month','DayOfWeek']).show()

# 8) Which flights covered maximum origin and destination by month wise


In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, desc,expr

#importing data from local data folder
df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)

#selecting the required columns , this makes processing faster
df_q2 = df.select('Origin','Month','Dest','TailNum').filter((df["TailNum"] != "0") & (df["TailNum"] != "000000"))

df_q2_no_null = df_q2.filter((df_q2["Origin"] != "null") & (df_q2["Dest"] != "null")) #removing rows with null values

print(type(df_q2))
print(" +++++ "*10)

most_frequent = df_q2_no_null.groupBy('Month','Origin','Dest','TailNum').agg(count("TailNum").alias("Total_trips")).sort(col("Total_trips").desc()).show()


<class 'pyspark.sql.dataframe.DataFrame'>
 +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++ 
+-----+------+----+-------+-----------+
|Month|Origin|Dest|TailNum|Total_trips|
+-----+------+----+-------+-----------+
|    8|   HNL| OGG| N841AL|         94|
|   12|   OGG| HNL| N841AL|         86|
|    7|   HNL| OGG| N841AL|         86|
|   12|   HNL| OGG| N841AL|         86|
|    8|   OGG| HNL| N485HA|         85|
|    7|   LGA| BOS| N913DE|         85|
|   10|   HNL| KOA| N655BR|         84|
|   10|   KOA| HNL| N655BR|         84|
|    7|   BOS| LGA| N913DE|         83|
|    1|   KOA| HNL| N646BR|         83|
|    1|   HNL| KOA| N646BR|         82|
|   10|   LGA| BOS| N911DE|         82|
|   11|   LGA| BOS| N916DE|         82|
|    5|   BOS| LGA| N908DE|         81|
|    7|   LIH| HNL| N841AL|         80|
|   10|   BOS| LGA| N911DE|         79|
|    5|   LGA| BOS| N908DE|         79|
|    5|   HNL| OGG| N836AL|         79|
|    8|   KOA| HNL| N655BR|         79|
|   11|

# 9) Average month wise arrival delay (flightnum wise)



In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, desc


df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)
df_no_cancel = df.filter('Cancelled== 0') 
df_q2 = df_no_cancel.select('Month','ArrDelay','FlightNum')



total_distance = df_q2.groupBy("Month", "FlightNum").agg(avg("ArrDelay").alias("avg_monthly_arrival_delay")).sort(['Month']).show()

+-----+---------+-------------------------+
|Month|FlightNum|avg_monthly_arrival_delay|
+-----+---------+-------------------------+
|    1|       59|        6.102941176470588|
|    1|      239|        6.021739130434782|
|    1|     1972|        6.168539325842697|
|    1|       96|       10.427631578947368|
|    1|     2543|        5.857142857142857|
|    1|     1670|        6.069767441860465|
|    1|     1170|        19.62439024390244|
|    1|     1839|       2.7282608695652173|
|    1|     1703|         9.09727626459144|
|    1|      318|        8.606666666666667|
|    1|     2592|        1.911764705882353|
|    1|     1185|        12.43661971830986|
|    1|     1430|       15.057692307692308|
|    1|     1799|        6.566666666666666|
|    1|     2759|        42.69565217391305|
|    1|     3058|       10.226666666666667|
|    1|     3014|       1.8641975308641976|
|    1|     2898|       0.7391304347826086|
|    1|     7109|       16.894736842105264|
|    1|     5448|       1.425925

# 10) Average month wise departure delay (flightnum wise)

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, desc


df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)
df_no_cancel = df.filter('Cancelled== 0') 
df_q2 = df_no_cancel.select('Month','DepDelay','FlightNum')



total_distance = df_q2.groupBy("Month","FlightNum").agg(avg("DepDelay").alias("avg_monthly_departure_delay")).sort(['Month']).show()

+-----+---------+---------------------------+
|Month|FlightNum|avg_monthly_departure_delay|
+-----+---------+---------------------------+
|    1|      739|         3.3737864077669903|
|    1|     2344|           7.40952380952381|
|    1|     2285|         -4.232558139534884|
|    1|     2847|          5.572327044025157|
|    1|      547|          17.09205020920502|
|    1|     1726|          5.578231292517007|
|    1|     2367|         3.8255813953488373|
|    1|     2478|       0.033707865168539325|
|    1|      847|         12.486842105263158|
|    1|      381|         4.7611940298507465|
|    1|      152|                  4.8515625|
|    1|      541|         12.436681222707424|
|    1|     2215|         1.4193548387096775|
|    1|     2682|          8.419354838709678|
|    1|     2250|         2.6056338028169015|
|    1|     1207|          9.425531914893616|
|    1|     2267|         31.345454545454544|
|    1|     2699|          5.810344827586207|
|    1|     7187|                 